In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import time
import datetime as dt
from bs4 import BeautifulSoup

The code used below was used the scrape the Reddit API that host all of reddit posts that have ever been

- This method was used in order to get as much data for our modeling as we can. 
- This code is not mine, it was Brian so I will explain how it was used. 

In [15]:
def query_pushshift(subreddit, kind='submission', skip=30, times=6, 
                    subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self'],
                    comfields = ['body', 'score', 'created_utc']): #Categorical fields in the website that we will scraping our data from. 

    stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=1000".format(kind, subreddit) # This will be where the website we wil be pulling from. 
    mylist = []
    
    for x in range(1, times):
        
        URL = "{}&after={}d".format(stem, skip * x) # This is used to format through the API to grab posts from different timeframes
        print(URL)
        response = requests.get(URL)  # this is the request pull to get the information from the website. 
        assert response.status_code == 200
        mine = response.json()['data'] #Scrapping the data from the api
        df = pd.DataFrame.from_dict(mine) # putting the data into a dataframe.
        mylist.append(df) #this is appending the data frame to the list. 
        time.sleep(2)
        
    full = pd.concat(mylist, sort=False)
    
    if kind == "submission": # if the target is identified as a submission the following fiels are entered
        
        full = full[subfield] # this will create a list of the all the subfields as a header.
        
        full = full.drop_duplicates() # this deletes duplicates from the field
        
        full = full.loc[full['is_self'] == True]
        
    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full["created_utc"].apply(get_date) # applying a timestamp. 
    
    full['timestamp'] = _timestamp

    print(full.shape)
    
    return full

In [16]:
query_pushshift("UnethicalLifeProTips").to_csv("./ULPT.csv")

https://api.pushshift.io/reddit/search/submission/?subreddit=UnethicalLifeProTips&size=1000&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=UnethicalLifeProTips&size=1000&after=60d
https://api.pushshift.io/reddit/search/submission/?subreddit=UnethicalLifeProTips&size=1000&after=90d
https://api.pushshift.io/reddit/search/submission/?subreddit=UnethicalLifeProTips&size=1000&after=120d
https://api.pushshift.io/reddit/search/submission/?subreddit=UnethicalLifeProTips&size=1000&after=150d
(4956, 9)


In [17]:
query_pushshift("LifeProTips").to_csv("./LPT.csv")

https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&size=1000&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&size=1000&after=60d
https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&size=1000&after=90d
https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&size=1000&after=120d
https://api.pushshift.io/reddit/search/submission/?subreddit=LifeProTips&size=1000&after=150d
(4977, 9)
